# Prerequisites

1. Create DataFrame

In [0]:
user_df = spark.read.csv(path="/FileStore/users_001.csv",
                         header=True,
                         inferSchema=True)
user_df.limit(4).display()                         

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
3,23,M,Writer,32067
4,26,M,technician,43537


# Basic Operations

In [0]:
user_df.show(n=10,truncate=False)
user_df.printSchema()

+---+---+---+-----------------------+------+
|id |age|gen|designation            |salary|
+---+---+---+-----------------------+------+
|1  |26 |M  |Technician             |85711 |
|2  |53 |F  |Other                  |94043 |
|3  |23 |M  |Writer                 |32067 |
|4  |26 |M  |Technician             |43537 |
|5  |33 |F  |Other                  |15213 |
|6  |42 |M  |Cheif Executive Officer|98101 |
|7  |57 |M  |Administrator          |91344 |
|8  |36 |M  |Administrator          |5201  |
|9  |29 |M  |Student                |1002  |
|10 |53 |M  |Lawyer                 |90703 |
+---+---+---+-----------------------+------+
only showing top 10 rows

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gen: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- salary: integer (nullable = true)



In [0]:
user_df.columns

Out[7]: ['id', 'age', 'gen', 'designation', 'salary']

## Selecting Columns

.select is a transformation

In [0]:
# df = user_df.select("id","gen")
# df.limit(4).display()

user_df.select("id","gen").limit(4).display()

id,gen
1,M
2,F
3,M
4,M


In [0]:
from pyspark.sql.functions import col

user_df.select(col("id").alias("user_id"), 
               col("gen").alias("gender")).limit(4).display()

user_id,gender
1,M
2,F
3,M
4,M


# Filter Operations

In [0]:
from pyspark.sql.functions import lower
user_df.filter(lower(col("designation"))=="technician").limit(4).show()

+---+---+---+-----------+------+
| id|age|gen|designation|salary|
+---+---+---+-----------+------+
|  1| 26|  M| Technician| 85711|
|  4| 26|  M| technician| 43537|
| 44| 26|  M| Technician| 46260|
| 77| 30|  M| Technician| 29379|
+---+---+---+-----------+------+



In [0]:
user_df.filter((col("designation") == "Technician") & (col("gen") == "M")).count()

Out[24]: 6

In [0]:
user_df.filter(col("designation").isin("Technician","Other")).display()

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
5,33,F,Other,15213
11,39,F,Other,30329
12,28,F,Other,6405
18,35,F,Other,37212
38,28,F,Other,54467
44,26,M,Technician,46260
77,30,M,Technician,29379
83,40,M,Other,44133


# Derive Columns

In [0]:
user_df.limit(4).display()

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
3,23,M,Writer,32067
4,26,M,technician,43537


In [0]:
from pyspark.sql.functions import lit
user_df.withColumn("salary_category", lit("Yes")).limit(4).display()

id,age,gen,designation,salary,is_alive
1,26,M,Technician,85711,Yes
2,53,F,Other,94043,Yes
3,23,M,Writer,32067,Yes
4,26,M,technician,43537,Yes


In [0]:
from pyspark.sql.functions import when
user_df.withColumn("salary_category", when(col("salary")>=80000,"High Paid")
                   .when((col("salary")>=50000) & (col("salary")<20000),"Medium Paid")
                   .otherwise("No Paid") 
                   ).limit(4).display()

id,age,gen,designation,salary,salary_category
1,26,M,Technician,85711,High Paid
2,53,F,Other,94043,High Paid
3,23,M,Writer,32067,No Paid
4,26,M,technician,43537,No Paid


In [0]:
user_df.selectExpr("*",
                   """
                   CASE
                   WHEN salary > 80000 THEN 'HIGH PAID'
                   WHEN salary > 20000 and salary < 80000 THEN 'Medium PAID'
                   ELSE 'No Paid' 
                   END AS salary_category
                   """).limit(4).display()

id,age,gen,designation,salary,salary_category
1,26,M,Technician,85711,HIGH PAID
2,53,F,Other,94043,HIGH PAID
3,23,M,Writer,32067,Medium PAID
4,26,M,technician,43537,Medium PAID


In [0]:
user_df.createOrReplaceGlobalTempView("user_gvw")

In [0]:
user_df.createOrReplaceTempView("user_vw")
spark.sql("""
          SELECT *, 
          CASE
                   WHEN salary > 80000 THEN 'HIGH PAID'
                   WHEN salary > 20000 and salary < 80000 THEN 'Medium PAID'
                   ELSE 'No Paid' 
                   END AS salary_category
                   FROM user_vw
          """).display()

id,age,gen,designation,salary,salary_category
1,26,M,Technician,85711,HIGH PAID
2,53,F,Other,94043,HIGH PAID
3,23,M,Writer,32067,Medium PAID
4,26,M,technician,43537,Medium PAID
5,33,F,Other,15213,No Paid
6,42,M,Cheif Executive Officer,98101,HIGH PAID
7,57,M,Administrator,91344,HIGH PAID
8,36,M,Administrator,5201,No Paid
9,29,M,Student,1002,No Paid
10,53,M,Lawyer,90703,HIGH PAID


# Grouping, Aggregations and Sorting

In [0]:
user_df.limit(4).display()

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
3,23,M,Writer,32067
4,26,M,technician,43537


In [0]:
user_df.groupBy("designation","gen").count().show(truncate=False)

+-----------------------+---+-----+
|designation            |gen|count|
+-----------------------+---+-----+
|technician             |M  |1    |
|Technician             |M  |6    |
|Programmer             |M  |20   |
|healthcare             |F  |1    |
|marketing              |M  |5    |
|Other                  |M  |10   |
|doctor                 |M  |1    |
|Artist                 |F  |5    |
|Writer                 |M  |8    |
|Lawyer                 |M  |4    |
|Administrator          |M  |15   |
|healthcare             |M  |3    |
|Cheif Executive Officer|M  |9    |
|Scientist              |F  |1    |
|Librarian              |M  |6    |
|Homemaker              |F  |2    |
|Other                  |F  |10   |
|Administrator          |F  |12   |
|Engineer               |M  |16   |
|Educator               |F  |9    |
+-----------------------+---+-----+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import sum

user_df.groupBy("designation").agg(sum("salary").alias("sum_salary")).sort(
    col("sum_salary"), ascending=False
).limit(4).display()

designation,sum_salary
Student,1762385
Administrator,1399976
Educator,1220858
Programmer,938773


In [0]:
from pyspark.sql.functions import sum

user_df.groupBy("designation").agg(sum("salary").alias("sum_salary")).sort(
    col("sum_salary").desc()
).limit(4).display()

designation,sum_salary
Student,1762385
Administrator,1399976
Educator,1220858
Programmer,938773


In [0]:
from pyspark.sql.functions import sum, avg,count
user_df.groupBy("designation").agg(sum("salary").alias("sum_salary"),
                                   avg("salary").alias("avg_salary")).display()

designation,sum_salary,avg_salary
Scientist,532883,53288.3
retired,149827,74913.5
Entertainment,142355,23725.833333333332
Student,1762385,45189.35897435898
Lawyer,194395,38879.0
Homemaker,138119,69059.5
Writer,451486,45148.6
marketing,289410,32156.666666666668
Programmer,938773,46938.65
Other,921002,46050.1


# Handling Missing / Null Records

## Prerequisites

In [0]:
employee_df = spark.read.csv(path="dbfs:/FileStore/synechron/employee-1.csv",
                             header=True,
                             inferSchema=True,
                             sep="|",
                             quote="'")
employee_df.display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
null,Bhavika,6,F,1987-01-22,Infosys,Team lead,2015-01-21,"Hadoop,PySpark,Kafka",null
105,Chitragandha,null,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
109,null,8,M,1987-01-22,null,Developer,2008-01-21,"C,C++,Java","1050000,1100000"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


In [0]:
employee_df.na.drop().display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"


In [0]:
employee_df.na.drop(subset=["col_id","col_name"]).display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
105,Chitragandha,null,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


In [0]:
employee_df.display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
null,Bhavika,6,F,1987-01-22,Infosys,Team lead,2015-01-21,"Hadoop,PySpark,Kafka",null
105,Chitragandha,null,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
109,null,8,M,1987-01-22,null,Developer,2008-01-21,"C,C++,Java","1050000,1100000"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


In [0]:
employee_df.na.fill("NULL IN SOURCE").na.fill(-1,subset=["col_id"]).na.fill(0,subset=["col_exp"]).display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
-1,Bhavika,6,F,1987-01-22,Infosys,Team lead,2015-01-21,"Hadoop,PySpark,Kafka",NULL IN SOURCE
105,Chitragandha,0,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
109,NULL IN SOURCE,8,M,1987-01-22,NULL IN SOURCE,Developer,2008-01-21,"C,C++,Java","1050000,1100000"
110,Vin,0,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


In [0]:
employee_df.printSchema()

root
 |-- col_id: integer (nullable = true)
 |-- col_name: string (nullable = true)
 |-- col_exp: integer (nullable = true)
 |-- col_gen: string (nullable = true)
 |-- col_dob: date (nullable = true)
 |-- col_company: string (nullable = true)
 |-- col_desig: string (nullable = true)
 |-- col_doj: date (nullable = true)
 |-- col_skills: string (nullable = true)
 |-- col_previous_expected_salary: string (nullable = true)



In [0]:
employee_df.columns

Out[65]: ['col_id',
 'col_name',
 'col_exp',
 'col_gen',
 'col_dob',
 'col_company',
 'col_desig',
 'col_doj',
 'col_skills',
 'col_previous_expected_salary']

In [0]:
employee_df.filter(col("col_id").is)

In [0]:
filtered_df = employee_df.filter(~(col("col_id").isNull() & col("col_previous_expected_salary").isNull()))
filtered_df.display()


col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
105,Chitragandha,null,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
109,null,8,M,1987-01-22,null,Developer,2008-01-21,"C,C++,Java","1050000,1100000"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


In [0]:
employee_df.filter(col("col_id").isNull() & col("col_previous_expected_salary").isNull()).display()

In [0]:
employee_df.display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_skills,col_previous_expected_salary
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"Hadoop,PySpark,Azure","1950000,1600000"
102,Acyuta,2,F,1987-03-29,TCS,Team Lead,2016-01-21,"C,C++,Java","1150000,1500000"
103,Anuvrata,1,M,1987-01-22,Infosys,Developer,2017-01-21,"Java,Python,Hadoop","1250000,1300000"
null,Bhavika,6,F,1987-01-22,Infosys,Team lead,2015-01-21,"Hadoop,PySpark,Kafka",null
105,Chitragandha,null,M,1987-01-22,CTS,Developer,null,"C,C++,Java","1200000,1400000"
106,Hritika,9,F,1987-01-22,Cisco,Developer,2016-01-21,"C,C++,Java","950000,1300000"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"Hadoop,PySpark,Kafka","200000,2100000"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"C,PySpark,Azure,AWS","3050000,2500000"
109,null,8,M,1987-01-22,null,Developer,2008-01-21,"C,C++,Java","1050000,1100000"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"Hadoop,PySpark,Kafka,Kafka","1550000,1400000"


# Working with Complex Types

In [0]:
from pyspark.sql.functions import split

result_df = (
    employee_df.withColumn("col_skills_temp", split(col("col_skills"), ","))
    .drop("col_skills")
    .withColumnRenamed("col_skills_temp", "col_skills")
)
result_df.printSchema()

root
 |-- col_id: integer (nullable = true)
 |-- col_name: string (nullable = true)
 |-- col_exp: integer (nullable = true)
 |-- col_gen: string (nullable = true)
 |-- col_dob: date (nullable = true)
 |-- col_company: string (nullable = true)
 |-- col_desig: string (nullable = true)
 |-- col_doj: date (nullable = true)
 |-- col_previous_expected_salary: string (nullable = true)
 |-- col_skills: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [0]:
from pyspark.sql.functions import array_contains
result_df.filter(array_contains(col("col_skills"),"PySpark")).display()

col_id,col_name,col_exp,col_gen,col_dob,col_company,col_desig,col_doj,col_previous_expected_salary,col_skills
101,Agastya,1,M,1987-01-22,Infosys,Developer,2015-01-21,"1950000,1600000","List(Hadoop, PySpark, Azure)"
null,Bhavika,6,F,1987-01-22,Infosys,Team lead,2015-01-21,null,"List(Hadoop, PySpark, Kafka)"
107,Jigyasa,12,M,1987-01-22,Infosys,Team Lead,2005-01-21,"200000,2100000","List(Hadoop, PySpark, Kafka)"
108,Kaveri,4,F,1987-01-22,cisco,R&D Engineer,2015-01-21,"3050000,2500000","List(C, PySpark, Azure, AWS)"
110,Vin,null,T,1987-01-22,CTS,Team Lead,2020-01-21,"1550000,1400000","List(Hadoop, PySpark, Kafka, Kafka)"
